# Logistic Regression: Banking Marketing Campaign

This notebook analyzes a banking marketing campaign dataset to predict whether customers will subscribe to a term deposit. We'll build and optimize a logistic regression model to classify customer responses.

In [ ]:
# Handle imports upfront
import pickle
import random
import warnings
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

## 1. Data acquisition

We'll start by loading the dataset from the provided URL and saving a local copy for future use.

### 1.1. Load the dataset

In [10]:
# Load the dataset from the provided URL
data_url = 'https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv'
data_df = pd.read_csv(data_url, sep=';')


### 1.2. Save local copy

In [11]:
# Make a directory for raw data
Path('../data/raw').mkdir(exist_ok=True, parents=True)

# Save a local copy of the raw data
data_df.to_parquet('../data/raw/bank-marketing-campaign-data.parquet')

### 1.3. Inspect

In [12]:
data_df.head().transpose()

,0,1,2,3,4
age,56,57,37,40,56
job,housemaid,services,services,admin.,services
marital,married,married,married,married,married
education,basic.4y,high.school,high.school,basic.6y,high.school
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day_of_week,mon,mon,mon,mon,mon


In [13]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

## 2. Data preparation

Before training our model, we need to prepare the data by splitting it into training and testing sets, and encoding categorical variables for use with scikit-learn.

### 2.1. Train-test split

In [14]:
# Split the data into training (75%) and testing (25%) sets
# This ensures we have unseen data to evaluate our final model
training_df, test_df = train_test_split(data_df, test_size=25)


### 2.2. Feature encoding

Machine learning algorithms work with numerical data, so we need to convert categorical variables (strings) to numerical format using ordinal encoding.

In [19]:
# Categorical features with 'object' datatypes (string) which need encoding
categorical_features = ['y','job','education','marital','default','housing','loan','contact','poutcome','day_of_week','month']

# Instantiate a encoder
encoder = OrdinalEncoder()

# Encode the categorical features in the training and testing datasets
training_df[categorical_features] = encoder.fit_transform(training_df[categorical_features])
test_df[categorical_features] = encoder.fit_transform(test_df[categorical_features])

# Inspect the result - there should be only float or int datatypes left
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41163 entries, 35070 to 32747
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41163 non-null  int64  
 1   job             41163 non-null  float64
 2   marital         41163 non-null  float64
 3   education       41163 non-null  float64
 4   default         41163 non-null  float64
 5   housing         41163 non-null  float64
 6   loan            41163 non-null  float64
 7   contact         41163 non-null  float64
 8   month           41163 non-null  float64
 9   day_of_week     41163 non-null  float64
 10  duration        41163 non-null  int64  
 11  campaign        41163 non-null  int64  
 12  pdays           41163 non-null  int64  
 13  previous        41163 non-null  int64  
 14  poutcome        41163 non-null  float64
 15  emp.var.rate    41163 non-null  float64
 16  cons.price.idx  41163 non-null  float64
 17  cons.conf.idx   41163 non-null  

## 3. Model training

We'll establish baseline performance using simple models, then build and optimize a logistic regression classifier.

In [20]:
# Empty dictionary to store performance results
results = {}

### 3.1. Random model performance

A random classifier serves as our weakest baseline - any useful model should significantly outperform random guessing.

In [21]:
# Generate random predictions for the testing set
# This serves as a baseline to compare our model performance against
predictions = [random.choice([0, 1]) for _ in range(len(test_df))]
# Calculate accuracy of random model
accuracy = accuracy_score(test_df['y'], predictions) * 100
# Store the accuracy in the results dictionary
results['Random'] = accuracy

print(f'Accuracy of random model: {accuracy:.2f}%')

Accuracy of random model: 44.00%


### 3.2. Constant 'no' model performance

Since this is a classification problem with imbalanced classes, we should check how well a model that always predicts the majority class would perform.

In [22]:
# Calculate accuracy if we always predict 'no' (the majority class)
accuracy = (test_df['y'].value_counts()[0]/len(test_df['y'])) * 100

# Store the accuracy in the results dictionary
results['Constant No'] = accuracy

print(f'Accuracy of constant "no" model: {accuracy:.2f}%')

Accuracy of constant "no" model: 96.00%


### 3.3. Logistic regression model performance

Now we'll train a basic logistic regression model with default parameters to see how much improvement we get over the baseline models.

In [ ]:
# Train a basic logistic regression model with default parameters

# Make predictions on the testing set

# Calculate accuracy of the test set predictions

# Store the accuracy in the results dictionary
results['Regression'] = accuracy

print(f"Testing accuracy: {accuracy:.2f}%")

### 3.4. Optimized logistic regression model performance

To get the best performance, we'll use grid search with cross-validation to find the optimal hyperparameters for our logistic regression model.

In [ ]:
# Define hyperparameters to search over
# These parameters can significantly affect model performance
hyperparameters = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],  # Different optimization algorithms
    'fit_intercept': [True, False],                 # Whether to include intercept term
    'max_iter': [50, 100, 200, 400, 800]            # Maximum iterations for convergence
}

# Use grid search with cross-validation to find best hyperparameters
# This systematically tests all combinations to find the optimal settings


# Save the best model and parameter combination
winning_parameters = grid.best_params_
winning_model = grid.best_estimator_

print(f'Best hyperparameters: {winning_parameters}')

In [ ]:
# Score the best model on the testing set

# Store the accuracy in the results dictionary
results['Optimized Regression'] = accuracy

print(f'Testing accuracy of optimized model: {accuracy:.2f}%')

### 3.5. Results

In [ ]:
# Create a bar plot to compare model performance

## 4. Final model

With the best hyperparameters identified, we'll train our final model and evaluate its performance on the test set. The confusion matrix will help us understand how well the model performs for each class.

### 4.1. Model re-training

In [ ]:
# Retrain model with winning hyperparameters on complete training set

# Calculate test set accuracy

print(f'Final model test set accuracy: {accuracy:.2f}%')

### 4.2. Model evaluation

In [ ]:
# Generate and display normalized confusion matrix
cm = confusion_matrix(testing_df['y'], predictions, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No', 'Yes'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - Test Set (Normalized)')
plt.show()

### 4.3. Save assets

#### 4.3.1. Data

In [ ]:
# Make sure the processed data directory exists
Path('../data/processed').mkdir(exist_ok=True, parents=True)

# Collect the training and testing datasets into a dictionary
datasets = {
    'training': training_df,
    'testing': testing_df
}

# Save the datasets to a file for future use
with open('../data/processed/datasets.pkl', 'wb') as datasets_file:
    pickle.dump(datasets, datasets_file)

#### 4.3.2. Models

In [ ]:
# Make sure the models directory exists
Path('../models/model.pkl').parent.mkdir(exist_ok=True, parents=True)

# Save the final model
with open('../models/model.pkl', 'wb') as output_file:
    pickle.dump(winning_model, output_file)